In [2]:
import code.utils.graph
import graph_tool.all as gt
import os
import ast
import re

print(os.getcwd())

def build_input_format(inputs_str):
    res = re.search('\[(.*)\]', inputs_str)
    return dict.fromkeys(res.group(1).split(','), 0) if res else {}
    

def build_graph_skeleton(g_str):
    g_elements = g_str.split('\n')
    g_name = g_elements[0].split(',')[1:]
    g_id = 0
    g_queuetime = 0
    
    g = gt.Graph(directed=True)
    g.gp['name'] = g.new_graph_property("string", g_name)
    g.gp['id'] = g.new_graph_property("string", str(g_id))
    g.gp['queue_time'] = g.new_graph_property("int", g_queuetime)
    g.gp['cur_stage'] = g.new_graph_property("int", -1)
    status = g.new_vertex_property("int")
    inputs = g.new_vertex_property("object")
    cache_runtime = g.new_vertex_property("int")
    remote_runtime = g.new_vertex_property("int")
    ops = g.new_vertex_property("vector<string>")

    # build vertices
    for el in g_elements[1:]:
        if el.startswith('v'):
            vid, inputs_str, operation = el.split(',')[1:]
            v = g.add_vertex()
            
            inputs[v] = build_input_format(inputs_str)
            cache_runtime[v] = 0
            remote_runtime[v] = 0 
            ops[v] = operation.split('|')
    
    # build edges
    for el in g_elements[1:]:
        if el.startswith('e'):
            v_src, v_dest = el.split(',')[1:]
            e = g.add_edge(v_src, v_dest)
    return g
            

def load_graph_skeleton(path):
    graph_skeletons = {}
    with open(path, 'r') as fd:
        graph_strs = fd.read().split('#')[1:]

        for g_str in graph_strs:
            g= build_graph_skeleton(g_str)
            graph_skeletons[g.gp.name] = g
    return graph_skeletons
            
path = "../tpch_pig_dags.g"
graph_skeletons = load_graph_skeleton(path)

print(graph_skeletons)


/local0/Kariz/expriments/macrobenchmark/ipython
{"['Q1', 'sequential']": <Graph object, directed, with 4 vertices and 3 edges at 0x7f6cf4cf8630>, "['Q2', 'sequential']": <Graph object, directed, with 8 vertices and 7 edges at 0x7f6cf4d05198>, "['Q3', 'sequential']": <Graph object, directed, with 6 vertices and 5 edges at 0x7f6cf4d05278>}


In [1]:
import code.utils.graph
import graph_tool.all as gt
import os
import ast
import re

import workload.sequential as sqw
import sys

print(os.getcwd())

config_file = "../config.json"

workload = sqw.Sequential(config_file)

print(workload.graph_skeleton_pool)

/local0/Kariz/expriments/macrobenchmark/ipython
{
  "type": "sequential",
  "graph_src": "file",
  "graph_skeleton_path": "/local0/Kariz/expriments/macrobenchmark/tpch_pig_dags.g",
  "stat_file_path": "/local0/Kariz/expriments/macrobenchmark/pig_tpch_stats",
  "input_dir": "s3a://data/pig-tpch/64G",
  "output_dir": "/tpch-64G-output",
  "benchmark_path": "/local0/Kariz/expriments/benchmark/BenchmarkScripts/tpch/pig"
}
{'Q1': <Graph object, directed, with 4 vertices and 3 edges at 0x7ff09e0a3198>, 'Q2': <Graph object, directed, with 8 vertices and 7 edges at 0x7ff09e0a3160>, 'Q3': <Graph object, directed, with 6 vertices and 5 edges at 0x7ff09e0a3940>}


In [4]:
# Load statistics
import utils.jobhistory as hist
import pandas as pd

print(workload.configs['stat_file_path'])

jobs_stats = pd.read_csv(workload.configs['stat_file_path'])

jobs_stats = jobs_stats[jobs_stats['runtime'] == 0].apply(hist.process_tasks, axis=1)

jobs_stats.to_csv(workload.configs['stat_file_path'], index=False, header=True)
#jobs = jobs.apply(process_tasks, axis=1)


/local0/Kariz/expriments/macrobenchmark/pig_tpch_stats
